In [ ]:
import random, string, urllib.request, json, getpass

#Generate root password
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

#Get public address and print connect command
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
  print(f'SSH command: ssh -p{port} root@{host}')

#Print root password
print(f'Root password: {password}')

Copy authtoken from https://dashboard.ngrok.com/auth
··········
SSH command: ssh -p15633 root@6.tcp.ngrok.io
Root password: JsnWCAfPP9SBwiT5qw0A


In [1]:
import argparse
from collections import defaultdict, namedtuple
from io import open
import math
import os
from random import shuffle, uniform
from datetime import datetime
from future.utils import iterkeys, iteritems
import torch

from future.builtins import range
from future.utils import iteritems
import pickle
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

In [2]:
# !pip install scipy
# !pip install matplotlib
# !pip install sklearn
!pip install torchsummaryX -q
from torchsummaryX import summary

In [3]:
import gc

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
#Get Glove Vectors
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip /content/glove.6B.zip

--2023-04-07 22:32:04--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-07 22:32:05--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove.6B.zip.1        3%[                    ]  29.30M  8.92MB/s    eta 86s    ^C
Archive:  /content/glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace glove.6B.100d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace glove.6B.20

In [5]:
#Prepare Glove Vectors
word2vec = {}
with open("/content/glove.6B.50d.txt", 'r', encoding="utf-8") as f:
    for l in f:
        line = l.split()
        word = line[0]
        vect = np.array(line[1:]).astype(float)
        word2vec[word] = vect
pickle.dump(word2vec, open(f'/content/6B.50_word2Vec.pkl', 'wb'))

# Data Loading Utilities

In [6]:
class InstanceData(object):
    """
    A bare-bones class to store the included properties of each instance. This is meant to act as easy access to the
    data, and provides a launching point for deriving your own features from the data.
    """
    def __init__(self, instance_properties):

        # Parameters specific to this instance
        self.instance_id = instance_properties['instance_id']
        self.token = instance_properties['token']
        self.part_of_speech = instance_properties['part_of_speech']
        self.morphological_features = instance_properties['morphological_features']
        self.dependency_label = instance_properties['dependency_label']
        self.dependency_edge_head = instance_properties['dependency_edge_head']

        # Derived parameters specific to this instance
        self.exercise_index = int(self.instance_id[8:10])
        self.token_index = int(self.instance_id[10:12])

        # Derived parameters specific to this exercise
        self.exercise_id = self.instance_id[:10]

        # Parameters shared across the whole session
        self.user = instance_properties['user']
        self.countries = instance_properties['countries']
        self.days = instance_properties['days']
        self.client = instance_properties['client']
        self.session = instance_properties['session']
        self.format = instance_properties['format']
        self.time = instance_properties['time']
        self.prompt = instance_properties.get('prompt', None)

        # Derived parameters shared across the whole session
        self.session_id = self.instance_id[:8]

    def to_features(self):
        """
        Prepares those features that we wish to use in the LogisticRegression example in this file. We introduce a bias,
        and take a few included features to use. Note that this dict restructures the corresponding features of the
        input dictionary, 'instance_properties'.

        Returns:
            to_return: a representation of the features we'll use for logistic regression in a dict. A key/feature is a
                key/value pair of the original 'instance_properties' dict, and we encode this feature as 1.0 for 'hot'.
        """
        to_return = dict()

        to_return['bias'] = 1.0
        to_return['user:' + self.user] = 1.0
        to_return['format:' + self.format] = 1.0
        to_return['token:' + self.token.lower()] = 1.0

        to_return['part_of_speech:' + self.part_of_speech] = 1.0
        for morphological_feature in self.morphological_features:
            to_return['morphological_feature:' + morphological_feature] = 1.0
        to_return['dependency_label:' + self.dependency_label] = 1.0
        
        time = datetime.now()
        if(time.second %10 == 0 and time.microsecond == 0):
          print(time)
          
        return to_return

In [7]:
#Load data from the file
def load_data(filename):
    """
    This method loads and returns the data in filename. If the data is labelled training data, it returns labels too.

    Parameters:
        filename: the location of the training or test data you want to load.

    Returns:
        data: a list of InstanceData objects from that data type and track.
        labels (optional): if you specified training data, a dict of instance_id:label pairs.
    """

    # 'data' stores a list of 'InstanceData's as values.
    data = []

    # If this is training data, then 'labels' is a dict that contains instance_ids as keys and labels as values.
    training = False
    if filename.find('train') != -1:
        training = True

    if training:
        labels = dict()

    num_exercises = 0
    print('Loading instances...')
    instance_properties = dict()

    with open(filename, 'rt') as f:
        for line in f:
            line = line.strip()

            # If there's nothing in the line, then we're done with the exercise. Print if needed, otherwise continue
            if len(line) == 0:
                num_exercises += 1
                if num_exercises % 100000 == 0:
                    print('Loaded ' + str(len(data)) + ' instances across ' + str(num_exercises) + ' exercises...')
                instance_properties = dict()

            # If the line starts with #, then we're beginning a new exercise
            elif line[0] == '#':
                if 'prompt' in line:
                    instance_properties['prompt'] = line.split(':')[1]
                else:
                    list_of_exercise_parameters = line[2:].split()
                    for exercise_parameter in list_of_exercise_parameters:
                        [key, value] = exercise_parameter.split(':')
                        if key == 'countries':
                            value = value.split('|')
                        elif key == 'days':
                            value = float(value)
                        elif key == 'time':
                            if value == 'null':
                                value = None
                            else:
                                assert '.' not in value
                                value = int(value)
                        instance_properties[key] = value

            # Otherwise we're parsing a new Instance for the current exercise
            else:
                line = line.split()
                if training:
                    assert len(line) == 7
                else:
                    assert len(line) == 6
                assert len(line[0]) == 12

                instance_properties['instance_id'] = line[0]

                instance_properties['token'] = line[1]
                instance_properties['part_of_speech'] = line[2]

                instance_properties['morphological_features'] = dict()
                for l in line[3].split('|'):
                    [key, value] = l.split('=')
                    if key == 'Person':
                        value = int(value)
                    instance_properties['morphological_features'][key] = value

                instance_properties['dependency_label'] = line[4]
                instance_properties['dependency_edge_head'] = int(line[5])
                if training:
                    label = float(line[6])
                    labels[instance_properties['instance_id']] = label
                data.append(InstanceData(instance_properties=instance_properties))

        print('Done loading ' + str(len(data)) + ' instances across ' + str(num_exercises) +
              ' exercises.\n')

    if training:
        return data, labels
    else:
        return data

In [8]:
def load_labels(filename):
    """
    This loads labels, either the actual ones or your predictions.

    Parameters:
        filename: the filename pointing to your labels

    Returns:
        labels: a dict of instance_ids as keys and labels between 0 and 1 as values
    """
    labels = dict()

    with open(filename, 'rt') as f:
        for line in f:
            line = line.strip()
            if len(line) == 0:
                continue
            else:
                line = line.split()
            instance_id = line[0]
            label = float(line[1])
            labels[instance_id] = label
    return labels


In [9]:
def compute_acc(actual, predicted):
    """
    Computes the accuracy of your predictions, using 0.5 as a cutoff.

    Note that these inputs are lists, not dicts; they assume that actual and predicted are in the same order.

    Parameters (here and below):
        actual: a list of the actual labels
        predicted: a list of your predicted labels
    """
    num = len(actual)
    acc = 0.
    for i in range(num):
        if round(actual[i], 0) == round(predicted[i], 0):
            acc += 1.
    acc /= num
    return acc


def compute_avg_log_loss(actual, predicted):
    """
    Computes the average log loss of your predictions.
    """
    num = len(actual)
    loss = 0.

    for i in range(num):
        p = predicted[i] if actual[i] > .5 else 1. - predicted[i]
        loss -= math.log(p)
    loss /= num
    return loss


def compute_auroc(actual, predicted):
    """
    Computes the area under the receiver-operator characteristic curve.
    This code a rewriting of code by Ben Hamner, available here:
    https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/auc.py
    """
    num = len(actual)
    temp = sorted([[predicted[i], actual[i]] for i in range(num)], reverse=True)

    sorted_predicted = [row[0] for row in temp]
    sorted_actual = [row[1] for row in temp]

    sorted_posterior = sorted(zip(sorted_predicted, range(len(sorted_predicted))))
    r = [0 for k in sorted_predicted]
    cur_val = sorted_posterior[0][0]
    last_rank = 0
    for i in range(len(sorted_posterior)):
        if cur_val != sorted_posterior[i][0]:
            cur_val = sorted_posterior[i][0]
            for j in range(last_rank, i):
                r[sorted_posterior[j][1]] = float(last_rank+1+i)/2.0
            last_rank = i
        if i==len(sorted_posterior)-1:
            for j in range(last_rank, i+1):
                r[sorted_posterior[j][1]] = float(last_rank+i+2)/2.0

    num_positive = len([0 for x in sorted_actual if x == 1])
    num_negative = num - num_positive
    sum_positive = sum([r[i] for i in range(len(r)) if sorted_actual[i] == 1])
    auroc = ((sum_positive - num_positive * (num_positive + 1) / 2.0) / (num_negative * num_positive))

    return auroc


def compute_f1(actual, predicted, cutoff = 0.5):
    """
    Computes the F1 score of your predictions. Note that we use 0.5 as the cutoff here.
    """
    num = len(actual)

    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negatives = 0

    for i in range(num):
        if actual[i] >= cutoff and predicted[i] >= cutoff:
            true_positives += 1
        elif actual[i] < cutoff and predicted[i] >= cutoff:
            false_positives += 1
        elif actual[i] >= cutoff and predicted[i] < cutoff:
            false_negatives += 1
        else:
            true_negatives += 1

    try:
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
        F1 = 2 * precision * recall / (precision + recall)
    except ZeroDivisionError:
        F1 = 0.0

    return F1


def evaluate_metrics(actual, predicted):
    """
    This computes and returns a dictionary of notable evaluation metrics for your predicted labels.
    """
    acc = compute_acc(actual, predicted)
    #avg_log_loss = compute_avg_log_loss(actual, predicted)
    auroc = compute_auroc(actual, predicted)
    F1 = compute_f1(actual, predicted)

    return  acc, auroc, F1 #avg_log_loss,  auroc, F1


def test_metrics():
    actual = [1, 0, 0, 1, 1, 0, 0, 1, 0, 1]
    predicted = [0.8, 0.2, 0.6, 0.3, 0.1, 0.2, 0.3, 0.9, 0.2, 0.7]
    metrics = evaluate_metrics(actual, predicted)
    metrics = {key: round(metrics[key], 3) for key in iterkeys(metrics)}
    assert metrics['accuracy'] == 0.700
    assert metrics['avglogloss'] == 0.613
    assert metrics['auroc'] == 0.740
    assert metrics['F1'] == 0.667
    print('Verified that our environment is calculating metrics correctly.')

# Load Data

In [10]:
training_data, training_labels = load_data("/content/en_es/en_es.slam.20190204.train")

Loading instances...
Loaded 317049 instances across 100000 exercises...
Loaded 635368 instances across 200000 exercises...
Loaded 951536 instances across 300000 exercises...
Loaded 1271940 instances across 400000 exercises...
Loaded 1591344 instances across 500000 exercises...
Loaded 1911212 instances across 600000 exercises...
Loaded 2227444 instances across 700000 exercises...
Loaded 2546704 instances across 800000 exercises...
Done loading 2622957 instances across 824012 exercises.



In [11]:
valid_data = load_data("/content/en_es/en_es.slam.20190204.dev")
valid_labels = load_labels("/content/en_es/en_es.slam.20190204.dev.key")

Loading instances...
Loaded 334439 instances across 100000 exercises...
Done loading 387374 instances across 115770 exercises.



In [ ]:
test_data = load_data("/content/en_es/en_es.slam.20190204.test")
test_labels = load_labels("/content/en_es/en_es.slam.20190204.test.key")

Loading instances...
Loaded 337728 instances across 100000 exercises...
Done loading 386604 instances across 114586 exercises.



In [12]:
min_days = float('inf')
max_days = -float('inf')
max_time = -float('inf')
min_time = float('inf')
for instance in training_data:
  days = instance.days
  time = instance.time
  if(time is None or time < 0):
    time = 0

  if(instance.days > max_days):
    max_days = instance.days
  if(instance.days < min_days):
    min_days = instance.days
  if(time > max_time):
    max_time = time
  if(time < min_time):
    min_time = time

In [13]:
min_days,max_days,max_time,min_time

(0.0, 28.042, 330554, 0)

# Pytorch Datasets

In [14]:
user_keyed_data = {}
user_keyed_label = {}

user_idx = 1
user2Idx = {"unk" : 0}
idx2User = {0: "unk"}
user_vocabulary = ["unk"]

token_idx = 1
word2Idx = {"unk" : 0}
token_vocabulary = ["unk"]


pos_idx = 1
pos2Idx = {"unk" : 0}
pos_vocabulary = ["unk"]

morph_idx = 1
morph2Idx = {"unk" : 0}
morph_vocab = ["unk"]

dep_label_idx = 1
depLabel2Idx = {"unk": 0}
depLabelVocab = ["unk"]

In [15]:
class ExcerciseDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels, sequence_size): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''
        global user_idx
        global user2Idx
        global user_vocabulary

        global token_idx
        global word2Idx
        global token_vocabulary

        global pos_idx
        global pos2Idx
        global pos_vocabulary

        global morph_idx
        global morph2Idx
        global morph_vocab

        global dep_label_idx
        global depLabel2Idx
        global depLabelVocab
        
        for i, instance in enumerate(data):
          user = instance.user
          if user not in user_keyed_data:
            user_keyed_data[user] = []
            user_keyed_label[user] = []
          
          exercise = []
          
          
          token = instance.token.lower()
          pos_tag = instance.part_of_speech.lower()
          morphology = instance.morphological_features
          dependency_label = instance.dependency_label.lower()
          time = instance.time
          if time is None or time < 0 or time > 100:
            time = 0
          days = instance.days
          if days is None or days > 100:
            days = 0
          days = float((days - min_days)/(max_days - min_days))
          time = float((time - min_time)/(max_time - min_time))
          if(days > 1):
            days = 1
          if(time > 1):
            time = 1

          label = labels[instance.instance_id]
          
          if user not in user2Idx:
            user2Idx[user] = user_idx
            idx2User[user_idx] = user
            user_vocabulary.append(user)
            user_idx += 1

          exercise.append(user2Idx[user])

          if token not in word2Idx:
            word2Idx[token] = token_idx
            token_vocabulary.append(token)
            token_idx += 1

          exercise.append(word2Idx[token])

          if pos_tag not in pos2Idx:
            pos2Idx[pos_tag] = pos_idx
            pos_vocabulary.append(pos_tag)
            pos_idx += 1

          exercise.append(pos2Idx[pos_tag])

          # morph_list = []
          # for morph_feature in morphology:
          #   if morph_feature not in self.morph2Idx:
          #     self.morph2Idx[morph_feature] = morph_idx
          #     self.morph_vocab.append(morph_feature)
          #     morph_idx += 1
          #   morph_list.append(self.morph2Idx[morph_feature])
          
          # exercise.append(morph_list)

          if dependency_label not in depLabel2Idx:
            depLabel2Idx[dependency_label] = dep_label_idx
            depLabelVocab.append(dependency_label)
            dep_label_idx += 1

          exercise.append(depLabel2Idx[dependency_label])
          exercise.append(float(days))
          exercise.append(float(time))

          user_keyed_data[user].append(torch.tensor(exercise, dtype = float))
          user_keyed_label[user].append(label)
          

        #At this point we have the exercises for each user.

        self.timesteped_data = []
        self.timesteped_labels = []

        #print(self.user_keyed_data[user])

        for user in user_keyed_data:
          for i in range(0, len(user_keyed_data[user]), 30):
            chunk = user_keyed_data[user][i:i + sequence_size]
            if(len(chunk) < 2):
              continue;
            self.timesteped_data.append(torch.stack(chunk,dim=0))
            self.timesteped_labels.append(torch.FloatTensor(user_keyed_label[user][i:i + sequence_size]))
        
        self.length = len(self.timesteped_data)
        
    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        data = self.timesteped_data[ind] # TODO
        labels = self.timesteped_labels[ind] # TODO
        return data, labels


    def collate_fn(batch):
        # batch of input mfcc coefficients
        batch_data_encoder = [x[0:len(x)//2] for x,y in batch] # TODO
        batch_data_decoder = [x[len(x)//2:] for x,y in batch] # TODO
        # batch of output phonemes
        batch_labels_encoder = [y[0:len(y)//2] for x,y in batch] # TODO
        batch_labels_decoder = [y[len(y)//2:] for x,y in batch] 

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_data_encoder_pad = pad_sequence(batch_data_encoder, batch_first=True, padding_value=0) # TODO
        batch_data_decoder_pad = pad_sequence(batch_data_decoder, batch_first=True, padding_value=0)

        encoder_lengths_data = [len(x) for x in batch_data_encoder] # TODO
        decoder_lengths_data = [len(x) for x in batch_data_decoder] 

        batch_labels_encoder_pad = pad_sequence(batch_labels_encoder, batch_first=True, padding_value=0) # TODO
        batch_labels_decoder_pad = pad_sequence(batch_labels_decoder, batch_first=True, padding_value=0)
        encoder_lengths_labels =  [len(x) for x in batch_labels_encoder] # TODO
        decoder_lengths_labels =  [len(x) for x in batch_labels_decoder] # TODO


        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_data_encoder_pad,batch_data_decoder_pad, batch_labels_encoder_pad,batch_labels_decoder_pad, torch.tensor(encoder_lengths_data), torch.tensor(decoder_lengths_data), torch.tensor(encoder_lengths_labels), torch.tensor(decoder_lengths_labels)

In [16]:
class ExcerciseValidationDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels, sequence_size=128): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        self.user_keyed_valid_data = {}
        self.user_keyed_valid_label = {}
        self.sequence_size = sequence_size

        global user_idx
        global user2Idx
        global user_vocabulary

        global token_idx
        global word2Idx
        global token_vocabulary

        global pos_idx
        global pos2Idx
        global pos_vocabulary

        global morph_idx
        global morph2Idx
        global morph_vocab

        global dep_label_idx
        global depLabel2Idx
        global depLabelVocab

        
        for i, instance in enumerate(data):
          user = instance.user
          if user not in self.user_keyed_valid_data:
            self.user_keyed_valid_data[user] = []
            self.user_keyed_valid_label[user] = []
          
          exercise = []
          
          
          token = instance.token.lower()
          pos_tag = instance.part_of_speech.lower()
          morphology = instance.morphological_features
          dependency_label = instance.dependency_label.lower()
          time = instance.time
          if time is None:
            time = 0
          days = instance.days
          if days is None:
            days = 0
          days = float((days - min_days)/(max_days - min_days))
          time = float((time - min_time)/(max_time - min_time))
          if(days > 1):
            print("days maxed")
            days = 1
          if(time > 1):
            print("time maxed")
            time = 1

          label = labels[instance.instance_id]

          assert user in user2Idx
          exercise.append(user2Idx[user])

          if token not in word2Idx:
            token = "unk"

          exercise.append(word2Idx[token])

          if pos_tag not in pos2Idx:
            pos_tag = "unk"

          exercise.append(pos2Idx[pos_tag])

          if dependency_label not in depLabel2Idx:
            dependency_label = "unk"

          exercise.append(depLabel2Idx[dependency_label])
          exercise.append(float(days))
          exercise.append(float(time))

          self.user_keyed_valid_data[user].append(torch.tensor(exercise, dtype = float))
          self.user_keyed_valid_label[user].append(label)
          

        #At this point we have the exercises for each user.

        self.timesteped_data = []
        self.timesteped_labels = []

        #print(self.user_keyed_data[user])

        for user in self.user_keyed_valid_data:
          for i in range(0, len(self.user_keyed_valid_data[user]), sequence_size):
            chunk1 = user_keyed_data[user][-sequence_size:] #History for the user
            assert len(chunk1) > 0
            chunk2 = self.user_keyed_valid_data[user][i:i + sequence_size]
            assert len(chunk2) > 0
            self.timesteped_data.append(torch.cat((torch.stack(chunk1,dim=0), torch.stack(chunk2,dim=0)), dim=0))
            self.timesteped_labels.append(torch.cat(
                (torch.FloatTensor(user_keyed_label[user][-sequence_size:]),
                torch.FloatTensor(self.user_keyed_valid_label[user][i:i + sequence_size])), 
                dim = 0
                ))
        
        self.length = len(self.timesteped_data)
        
    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        data = self.timesteped_data[ind] # TODO
        labels = self.timesteped_labels[ind] # TODO
        return data, labels


    def collate_fn(batch):
        # batch of input mfcc coefficients
        
        batch_data_encoder = [] # TODO
        batch_data_decoder = [] # TODO

        # batch of output phonemes
        batch_labels_encoder = [] # TODO
        batch_labels_decoder = [] 

        for x,y in batch:
          user_id =  idx2User[int(x[0][0].item())]
          sequence_size = 128
          if(len(user_keyed_data[user_id]) < sequence_size):
            sequence_size = len(user_keyed_data[user_id])

          batch_data_encoder.append(x[0:sequence_size])
          assert len(x[0:sequence_size]) > 0
          batch_data_decoder.append(x[sequence_size:])
          assert len(x[sequence_size:]) > 0
          batch_labels_encoder.append(y[0:sequence_size])
          assert len(y[0:sequence_size]) > 0
          batch_labels_decoder.append(y[sequence_size:])
          assert len(y[sequence_size:]) > 0

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_data_encoder_pad = pad_sequence(batch_data_encoder, batch_first=True, padding_value=0) # TODO
        batch_data_decoder_pad = pad_sequence(batch_data_decoder, batch_first=True, padding_value=0)

        encoder_lengths_data = [len(x) for x in batch_data_encoder] # TODO
        decoder_lengths_data = [len(x) for x in batch_data_decoder] 

        batch_labels_encoder_pad = pad_sequence(batch_labels_encoder, batch_first=True, padding_value=0) # TODO
        batch_labels_decoder_pad = pad_sequence(batch_labels_decoder, batch_first=True, padding_value=0)
        encoder_lengths_labels =  [len(x) for x in batch_labels_encoder] # TODO
        decoder_lengths_labels =  [len(x) for x in batch_labels_decoder] # TODO


        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_data_encoder_pad,batch_data_decoder_pad, batch_labels_encoder_pad,batch_labels_decoder_pad, torch.tensor(encoder_lengths_data), torch.tensor(decoder_lengths_data), torch.tensor(encoder_lengths_labels), torch.tensor(decoder_lengths_labels)

In [17]:
train_data = ExcerciseDataset(training_data, training_labels, 256) #TODO
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data, 
    num_workers = 8,
    batch_size  = 64, 
    pin_memory  = True,
    shuffle     = True,
    collate_fn = ExcerciseDataset.collate_fn
) #TODO

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [18]:
val_dataset = ExcerciseValidationDataset(valid_data, valid_labels, 128) #TODO

days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed
days maxed

In [19]:
val_loader = torch.utils.data.DataLoader(
    dataset     = val_dataset, 
    num_workers = 8,
    batch_size  = 64, 
    pin_memory  = True,
    shuffle     = True,
    collate_fn = ExcerciseValidationDataset.collate_fn
) #TODO

In [20]:
# sanity check
i = 0;

for data in val_loader:
    x_encoder,x_decoder, y_encoder,y_decoder, lx_encoder,lx_decoder, ly_encoder, ly_decoder = data
    print(x_encoder.shape,x_decoder.shape, y_encoder.shape,y_decoder.shape, lx_encoder.shape, lx_decoder.shape,ly_encoder.shape, ly_decoder.shape)
    i += 1
    if(i==2):
      break 

torch.Size([64, 128, 6]) torch.Size([64, 128, 6]) torch.Size([64, 128]) torch.Size([64, 128]) torch.Size([64]) torch.Size([64]) torch.Size([64]) torch.Size([64])
torch.Size([64, 128, 6]) torch.Size([64, 128, 6]) torch.Size([64, 128]) torch.Size([64, 128]) torch.Size([64]) torch.Size([64]) torch.Size([64]) torch.Size([64])


# Encoder

In [21]:
#Prepare GloVe for encoder
matrix_len = len(token_vocabulary)
weights_matrix = np.zeros((matrix_len, 50))
words_found = 0

for i, word in enumerate(token_vocabulary):
    try: 
        weights_matrix[i] = word2vec[word]
        words_found += 1
    except KeyError:
      if(i == 0):
        print("here")
        weights_matrix[i] = np.zeros((50, ))
      else:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(50, ))

In [102]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, token_embedding_weight_matrix, encoder_hidden_size):
        super(Encoder, self).__init__()

        self.user_embedding = torch.nn.Embedding(len(user_vocabulary), 10)
        self.token_embedding = torch.nn.Embedding(len(token_vocabulary), 15)
        #self.token_embedding.load_state_dict({'weight': torch.as_tensor(token_embedding_weight_matrix, dtype=float)})
        #self.token_embedding.weight.requires_grad = True

        self.pos_embedding = torch.nn.Embedding(len(pos_vocabulary), 10)
        self.dependency_embedding = torch.nn.Embedding(len(depLabelVocab), 10)
        self.label_embedding = torch.nn.Embedding(2, 10)

        self.lstm1 = torch.nn.LSTM(input_size = 57, hidden_size = encoder_hidden_size, num_layers = 2, bidirectional = True, batch_first = True, dropout = 0.1)

    def forward(self, x, x_lens, labels):
        # Where are x and x_lens coming from? The dataloader
        #TODO: Call the embedding layer
        with torch.cuda.amp.autocast():
          user_embeddings = self.user_embedding(x[:,:,0].clone().detach().to(torch.int64))
          token_embeddings = self.token_embedding(x[:,:,1].clone().detach().to(torch.int64))
          pos_embeddings = self.pos_embedding(x[:,:,2].clone().detach().to(torch.int64))
          dependency_embeddings = self.dependency_embedding(x[:,:,3].clone().detach().to(torch.int64))
          label_embeddings = self.label_embedding(labels.clone().detach().to(torch.int64))

          concatenated_out = torch.cat((
              user_embeddings.clone().detach().to(torch.float),
              token_embeddings.clone().detach().to(torch.float),
              pos_embeddings.clone().detach().to(torch.float),
              dependency_embeddings.clone().detach().to(torch.float),
              x[:,:,4].reshape(x[:,:,4].shape[0], x[:,:,4].shape[1], 1).clone().detach().to(torch.float),
              x[:,:,5].reshape(x[:,:,5].shape[0], x[:,:,5].shape[1], 1).clone().detach().to(torch.float), 
              label_embeddings.clone().detach().to(torch.float)), dim=2)
        
          packed_out = pack_padded_sequence(concatenated_out, x_lens, batch_first = True, enforce_sorted=False)
          out = self.lstm1(packed_out)[0]
          #out = self.pBLSTMs(out)
          # TODO: Pad Packed Sequence
          encoder_outputs, encoder_lens = pad_packed_sequence(out, batch_first = True)
          
          # Remember the number of output(s) each function returns

        return encoder_outputs, encoder_lens

In [58]:
encoder = Encoder(weights_matrix, 16)

In [69]:
for i, data in enumerate(val_loader):
  x_encoder,x_decoder, y_encoder,y_decoder, lx_encoder,lx_decoder, ly_encoder, ly_decoder = data
  print(type((x_encoder[:,:,3].reshape(x_encoder[:,:,3].shape[0], x_encoder[:,:,3].shape[1], 1)[0,0].item())))
  encoder_outputs, encoder_lens = encoder(x_encoder, lx_encoder, y_encoder)
  print(encoder_outputs.shape)
  break
  # encoder_out_new = torch.zeros((encoder_outputs.shape[0], encoder_outputs.shape[2]))
  # for batch_output_index in range(encoder_outputs.shape[0]):
  #   encoder_out_new[batch_output_index] = torch.sum(encoder_outputs[batch_output_index,0:encoder_lens[batch_output_index]], dim = 0)
  # print(encoder_out_new.shape)
  #print(torch.sum(encoder_outputs[:,:encoder_lens], dim = 1).shape)

<class 'float'>
torch.Size([64, 4639, 32])


# Decoder

In [103]:
class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [107]:
class Decoder(torch.nn.Module):

    def __init__(self,token_embedding_weight_matrix, decoder_hidden_size, output_size = 1):
        super().__init__()

        self.user_embedding = torch.nn.Embedding(len(user_vocabulary), 10)
        self.token_embedding = torch.nn.Embedding(len(token_vocabulary), 15)
        #self.token_embedding.load_state_dict({'weight': torch.as_tensor(token_embedding_weight_matrix)})
        #self.token_embedding.weight.requires_grad = True

        self.pos_embedding = torch.nn.Embedding(len(pos_vocabulary), 10)
        self.dependency_embedding = torch.nn.Embedding(len(depLabelVocab), 10)

        self.lstm1 = torch.nn.GRU(input_size = 47, hidden_size = decoder_hidden_size, num_layers = 2, bidirectional = True, batch_first = True, dropout = 0.1)
        self.mlp_size = 2*decoder_hidden_size + decoder_hidden_size
        self.mlp = torch.nn.Sequential(
            PermuteBlock(), 
            torch.nn.BatchNorm1d(self.mlp_size), 
            PermuteBlock(),

            torch.nn.Linear(self.mlp_size, self.mlp_size//2),
            PermuteBlock(), 
            torch.nn.BatchNorm1d(self.mlp_size//2), 
            PermuteBlock(),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.1),

            torch.nn.Linear(self.mlp_size//2, self.mlp_size//4),
            PermuteBlock(), 
            torch.nn.BatchNorm1d(self.mlp_size//4), 
            PermuteBlock(),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.1),

            torch.nn.Linear(self.mlp_size//4, self.mlp_size//8),
            PermuteBlock(), 
            torch.nn.BatchNorm1d(self.mlp_size//8), 
            PermuteBlock(),
            torch.nn.GELU(),
            torch.nn.Dropout(p = 0.1),

            torch.nn.Linear(self.mlp_size//8, output_size)#,

            #torch.nn.Sigmoid()
        )
        
        #self.sigmoid = torch.nn.Sigmoid(dim=2)

    def forward(self, encoder_out, x, x_lens):
        #TODO call your MLP
        #TODO Think what should be the final output of the decoder for the classification 

        #Assuming encoder_out is of size BatchSize, H_out
        user_embeddings = self.user_embedding(x[:,:,0].clone().detach().to(torch.int64))
        token_embeddings = self.token_embedding(x[:,:,1].clone().detach().to(torch.int64))
        pos_embeddings = self.pos_embedding(x[:,:,2].clone().detach().to(torch.int64))
        dependency_embeddings = self.dependency_embedding(x[:,:,3].clone().detach().to(torch.int64))

        concatenated_out = torch.cat((
            user_embeddings.clone().detach().to(torch.float),
            token_embeddings.clone().detach().to(torch.float),
            pos_embeddings.clone().detach().to(torch.float),
            dependency_embeddings.clone().detach().to(torch.float),
            x[:,:,4].reshape(x[:,:,4].shape[0], x[:,:,4].shape[1], 1).clone().detach().to(torch.float),
            x[:,:,5].reshape(x[:,:,5].shape[0], x[:,:,5].shape[1], 1).clone().detach().to(torch.float)), dim=2)

        packed_out = pack_padded_sequence(concatenated_out, x_lens, batch_first = True, enforce_sorted=False)

        out = self.lstm1(packed_out)[0]
        decoder_output, decoder_lens = pad_packed_sequence(out, batch_first = True)

        out = self.mlp(torch.cat((decoder_output, encoder_out.reshape(encoder_out.shape[0],1, encoder_out.shape[1]).repeat(1,decoder_output.shape[1],1)), dim = 2))
        return out.reshape(out.shape[0],out.shape[1])

In [105]:
decoder = Decoder(weights_matrix, 32, 1)

In [108]:
for i, data in enumerate(val_loader):
  x_encoder,x_decoder,y_encoder,y_decoder,lx_encoder,lx_decoder, ly_encoder, ly_decoder = data
  print(x_encoder,x_decoder,y_encoder,y_decoder,lx_encoder,lx_decoder, ly_encoder, ly_decoder)

  #print(encoder(x_encoder, lx_encoder, y_encoder)[0].shape)
  encoder_out, encoder_lens = encoder(x_encoder, lx_encoder, y_encoder)

  encoder_out_new = torch.zeros((encoder_out.shape[0], encoder_out.shape[2]))

  for batch_output_index in range(encoder_out.shape[0]):
    encoder_out_new[batch_output_index] = torch.sum(encoder_out[batch_output_index,0:encoder_lens[batch_output_index]], dim = 0)

  decoder_out  = decoder(encoder_out_new, x_decoder, lx_decoder)
  print(decoder_out.shape)
  break

tensor([[[2.0680e+03, 1.5400e+02, 4.0000e+00, 4.0000e+00, 7.1543e-01,
          1.5126e-05],
         [2.0680e+03, 1.1000e+01, 1.0000e+00, 1.0000e+00, 7.1543e-01,
          2.4202e-05],
         [2.0680e+03, 9.0000e+00, 2.0000e+00, 2.0000e+00, 7.1543e-01,
          2.4202e-05],
         ...,
         [2.0680e+03, 4.6100e+02, 4.0000e+00, 4.0000e+00, 7.4831e-01,
          2.1177e-05],
         [2.0680e+03, 7.8000e+01, 3.0000e+00, 3.0000e+00, 7.4831e-01,
          1.8151e-05],
         [2.0680e+03, 1.2090e+03, 4.0000e+00, 4.0000e+00, 7.4831e-01,
          1.8151e-05]],

        [[6.5200e+02, 7.8000e+01, 3.0000e+00, 3.0000e+00, 5.3809e-01,
          8.1681e-05],
         [6.5200e+02, 1.8500e+02, 4.0000e+00, 1.0000e+00, 5.3809e-01,
          8.1681e-05],
         [6.5200e+02, 9.0000e+00, 2.0000e+00, 2.0000e+00, 5.3809e-01,
          8.1681e-05],
         ...,
         [6.5200e+02, 7.1800e+02, 4.0000e+00, 4.0000e+00, 6.7709e-01,
          2.5714e-04],
         [6.5200e+02, 2.9000e+01, 8.0000

RuntimeError: ignored

# Complete Model

In [109]:
class KnowledgeModel(torch.nn.Module):

    def __init__(self, embed_size=128, output_size=1):
        super().__init__()

        #Prepare GloVe for encoder
        matrix_len = len(token_vocabulary)
        weights_matrix = np.zeros((matrix_len, 50))
        words_found = 0

        for i, word in enumerate(token_vocabulary):
          try: 
              weights_matrix[i] = word2vec[word]
              words_found += 1
          except KeyError:
            if(i == 0):
              weights_matrix[i] = np.zeros((50, ))
            else:
              weights_matrix[i] = np.random.normal(scale=0.6, size=(50, ))

        self.encoder        = Encoder(weights_matrix, embed_size) # TODO: Initialize Encoder
        self.decoder        = Decoder(weights_matrix, 2*embed_size, output_size) # TODO: Initialize Decoder 
    
    def forward(self, x_encoder,x_encoder_lengths, y_encoder_labels, x_decoder, x_decoder_lengths):
        encoder_out, encoder_lens = self.encoder(x_encoder, x_encoder_lengths, y_encoder_labels)
        encoder_out_new = torch.zeros((encoder_out.shape[0], encoder_out.shape[2]))
        for batch_output_index in range(encoder_out.shape[0]):
          encoder_out_new[batch_output_index] = torch.mean(encoder_out[batch_output_index,0:encoder_lens[batch_output_index]], dim = 0)
        #print(encoder_out_new.shape)
        encoder_out_new = encoder_out_new.to(device)
        decoder_out  = self.decoder(encoder_out_new, x_decoder,  x_decoder_lengths)

        return decoder_out

In [110]:
model = KnowledgeModel(
    embed_size  = 512,
    output_size = 1
).to(device)
print(model)
#summary(model,x_encoder.to(device),lx_encoder,y_encoder.to(device), x_decoder.to(device),lx_decoder)

KnowledgeModel(
  (encoder): Encoder(
    (user_embedding): Embedding(2594, 10)
    (token_embedding): Embedding(1968, 15)
    (pos_embedding): Embedding(17, 10)
    (dependency_embedding): Embedding(42, 10)
    (label_embedding): Embedding(2, 10)
    (lstm1): LSTM(57, 512, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  )
  (decoder): Decoder(
    (user_embedding): Embedding(2594, 10)
    (token_embedding): Embedding(1968, 15)
    (pos_embedding): Embedding(17, 10)
    (dependency_embedding): Embedding(42, 10)
    (lstm1): GRU(47, 1024, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (mlp): Sequential(
      (0): PermuteBlock()
      (1): BatchNorm1d(3072, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PermuteBlock()
      (3): Linear(in_features=3072, out_features=1536, bias=True)
      (4): PermuteBlock()
      (5): BatchNorm1d(1536, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): PermuteB

# **Training**

In [111]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler =  torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=3, mode='min') # fill this out
criterion = torch.nn.BCEWithLogitsLoss(reduction = 'none' ,pos_weight = torch.tensor([10]).to(device))
# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

In [112]:
def create_loss_mask(lens):
    mask = torch.arange(max(lens))
    mask = torch.tile(mask, (len(lens), 1))
    t = torch.tile( lens.reshape((len(lens), 1)) , (1, mask.shape[1]))
    mask = mask < t
    return mask

In [113]:
def macro_double_soft_f1(y, y_hat, reduction='mean'): # Written in PyTorch
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.
    This version uses the computation of soft-F1 for both positive and negative class for each label.
    Args:
        y (torch.FloatTensor): targets array of shape (BATCH_SIZE, N_LABELS), including 0. and 1.
        y_hat (torch.FloatTensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
    Returns:
        cost (scalar): value of the cost function for the batch
    """

    # dtype = y_hat.dtype
    # y = y.to(dtype)

    # FloatTensor = torch.cuda.FloatTensor
    # y = FloatTensor(y)
    # y_hat = FloatTensor(y_hat)


    tp = (y_hat * y).sum(dim=0) # soft
    fp = (y_hat * (1-y)).sum(dim=0) # soft
    fn = ((1-y_hat) * y).sum(dim=0) # soft
    tn = ((1-y_hat) * (1-y)).sum(dim=0) # soft

    soft_f1_class1 = 2*tp / (2*tp + fn + fp + 1e-16)
    soft_f1_class0 = 2*tn / (2*tn + fn + fp + 1e-16)
    cost_class1 = 1 - soft_f1_class1 # reduce 1 - soft-f1_class1 in order to increase soft-f1 on class 1
    cost_class0 = 1 - soft_f1_class0 # reduce 1 - soft-f1_class0 in order to increase soft-f1 on class 0
    cost = 0.5 * (cost_class1 + cost_class0) # take into account both class 1 and class 0

    if reduction == 'none':
        return cost

    if reduction == 'mean':
        macro_cost = cost.mean()
        return macro_cost

In [114]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):
    
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    total_loss = 0
    total_f1 = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x_encoder,x_decoder,y_encoder,y_decoder,lx_encoder,lx_decoder, ly_encoder, ly_decoder = data
        x_encoder,x_decoder,y_encoder,y_decoder = x_encoder.to(device),x_decoder.to(device),y_encoder.to(device),y_decoder.to(device)
        #x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():     
            decoder_out = model(x_encoder,lx_encoder,y_encoder,x_decoder,lx_decoder)
            #print(h.shape)
            #print(lh)
            loss = criterion(decoder_out, y_decoder)
            loss_mask = create_loss_mask(ly_decoder)
            loss_mask = loss_mask.to(device)
            masked_loss = loss * loss_mask

            assert torch.sum(loss_mask) != 0
            loss = torch.sum(masked_loss)/torch.sum(loss_mask)


        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16. 
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x_encoder,x_decoder,y_encoder,y_decoder,lx_encoder,lx_decoder, ly_encoder, ly_decoder, decoder_out, loss 
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar
    
    return total_loss / len(train_loader)


def validate_model(model, val_loader):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    
    vdist = 0

    for i, data in enumerate(val_loader):

        x_encoder,x_decoder,y_encoder,y_decoder,lx_encoder,lx_decoder, ly_encoder, ly_decoder = data
        x_encoder,x_decoder,y_encoder,y_decoder = x_encoder.to(device),x_decoder.to(device),y_encoder.to(device),y_decoder.to(device)

        #x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():   
          with torch.inference_mode():
              decoder_out = model(x_encoder,lx_encoder,y_encoder,x_decoder,lx_decoder)
              #h = torch.permute(h, (1, 0, 2))
              loss = criterion(decoder_out, y_decoder)
              loss_mask = create_loss_mask(ly_decoder)
              loss_mask = loss_mask.to(device)
              masked_loss = loss * loss_mask

              loss = torch.sum(masked_loss)/torch.sum(loss_mask)

        total_loss += float(loss.item())

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()
    
        del x_encoder,x_decoder,y_encoder,y_decoder,lx_encoder,lx_decoder, ly_encoder, ly_decoder,decoder_out, loss
        torch.cuda.empty_cache()
        
    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    return total_loss

In [115]:
def get_eval_results(model, val_loader):
  results = []
  sigmoid = torch.nn.Sigmoid()
  model.eval()
  batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
  for i, data in enumerate(val_loader):

      x_encoder,x_decoder,y_encoder,y_decoder,lx_encoder,lx_decoder, ly_encoder, ly_decoder = data
      x_encoder,x_decoder,y_encoder,y_decoder = x_encoder.to(device),x_decoder.to(device),y_encoder.to(device),y_decoder.to(device)

      with torch.cuda.amp.autocast(): 
        with torch.no_grad():
            decoder_out = model(x_encoder,lx_encoder,y_encoder,x_decoder,lx_decoder)
            decoder_out = sigmoid(decoder_out)

      #prediction_string = decode_prediction(h, lh, decoder, PHONEME_MAP=LABELS) # TODO call decode_prediction 
      #TODO save the output in results array.
      i = 0
      for out in decoder_out.detach().cpu().numpy():
        out1 = out[0:lx_decoder[i]]
        results.extend(out1)
        i+=1
      
      del x_encoder,x_decoder,y_encoder,y_decoder,lx_encoder,lx_decoder, ly_encoder, ly_decoder,decoder_out 
      torch.cuda.empty_cache()
      batch_bar.update()
  
  batch_bar.close()
  return results

In [116]:
actual = []
for instance_id in iterkeys(valid_labels):
  try:
      actual.append(valid_labels[instance_id])
  except KeyError:
      print('No prediction for instance ID ' + instance_id + '!')

In [117]:
torch.cuda.empty_cache()
gc.collect()

for epoch in range(0, 50):

    print("\nEpoch: {}/{}".format(epoch+1, 50))
    
    curr_lr = optimizer.param_groups[0]['lr']

    train_loss              = train_model(model, train_loader, criterion, optimizer) 
    valid_loss  = validate_model(model, val_loader)
    val_results = get_eval_results(model, val_loader)
    
    scheduler.step(valid_loss)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("Val Loss {:.04f}".format(valid_loss))
    print(evaluate_metrics(actual, val_results))


    # wandb.log({
    #     'train_loss': train_loss,  
    #     'valid_dist': valid_dist, 
    #     'valid_loss': valid_loss, 
    #     'lr'        : curr_lr
    # })
    
#     save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
#     wandb.save(epoch_model_path)
#     print("Saved epoch model")

#     if valid_dist <= best_lev_dist:
#         best_lev_dist = valid_dist
#         save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
#         wandb.save(best_model_path)
#         print("Saved best model")
#       # You may find it interesting to exlplore Wandb Artifcats to version your models
# run.finish()


Epoch: 1/50


	Train Loss 1.2265	 Learning Rate 0.0010000
Val Loss 1.2848
(0.5192578748186507, 0.5058548346376416, 0.22332080064692006)

Epoch: 2/50


	Train Loss 0.9813	 Learning Rate 0.0010000
Val Loss 1.7580
(0.5933619706020539, 0.49847163872508576, 0.203963286810428)

Epoch: 3/50


	Train Loss 0.7367	 Learning Rate 0.0010000
Val Loss 2.3813
(0.6398416001073898, 0.49977628393473367, 0.1941392152900078)

Epoch: 4/50


	Train Loss 0.6436	 Learning Rate 0.0010000
Val Loss 2.6450
(0.6551627109718257, 0.5000939175601892, 0.19021877461972)

Epoch: 5/50


	Train Loss 0.6412	 Learning Rate 0.0010000
Val Loss 2.4277
(0.6345082530061388, 0.49891146253325636, 0.19524502227475227)

Epoch: 6/50


	Train Loss 0.5210	 Learning Rate 0.0005000
Val Loss 3.7230
(0.6872892863227785, 0.49857041217505405, 0.17884250474383306)

Epoch: 7/50


Train:  26%|██▌       | 358/1385 [01:29<04:10,  4.09it/s, loss=0.4075, lr=0.000500]

KeyboardInterrupt: ignored

In [78]:
val_results = get_eval_results(model, val_loader)

In [38]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
results = []
sigmoid = torch.nn.Sigmoid()
model.eval()
for data in tqdm(val_loader):

    x_encoder,x_decoder,y_encoder,y_decoder,lx_encoder,lx_decoder, ly_encoder, ly_decoder = data
    x_encoder,x_decoder,y_encoder,y_decoder = x_encoder.to(device),x_decoder.to(device),y_encoder.to(device),y_decoder.to(device)

    with torch.no_grad():
        decoder_out = model(x_encoder,lx_encoder,y_encoder,x_decoder,lx_decoder)
        decoder_out = sigmoid(decoder_out)

    #prediction_string = decode_prediction(h, lh, decoder, PHONEME_MAP=LABELS) # TODO call decode_prediction 
    #TODO save the output in results array.
    i = 0
    for out in decoder_out.detach().cpu().numpy():
      out1 = out[0:lx_decoder[i]]
      results.extend(out1)
      i+=1
    
    del x_encoder,x_decoder,y_encoder,y_decoder,lx_encoder,lx_decoder, ly_encoder, ly_decoder,decoder_out 
    torch.cuda.empty_cache()

Extra


In [ ]:
# class ExcerciseToken(object):
#   def __init__(self, token, id, pos_tag, morphological_features, dep_label, dep_edge_head):
#     self.token = token
#     self.id = id
#     self.pos_tag = pos_tag
#     self.morphological_features = morphological_features
#     self.dep_label = dep_label
#     self.dep_edge_head = dep_edge_head
  
#   def set_label(self, label):
#     self.label = label


In [ ]:
# class ExcerciseInformation(object):
#   def __init__(self):
#     self.exercise_tokens = []
#     self.prompt = ""
#     self.user = ""
#     self.countries = ""
#     self.days = 0.0
#     self.client = ""
#     self.session = ""
#     self.format = ""
#     self.time = 0.0
  
#   def add_exercise_token(self, excercise_token_info: ExcerciseToken):
#     self.exercise_tokens.append(excercise_token_info)
  
#   def set_prompt(self, prompt: str):
#     self.prompt = prompt

#   def set_user(self, user: str):
#     self.user = user
  
#   def set_countries(self, countries: str):
#     self.countries = countries

#   def set_days(self, days: float):
#     self.days = days
  
#   def set_client(self, client: str):
#     self.client = client
  
#   def set_session(self, session: str):
#     self.session = session
  
#   def set_format(self, format: str):
#     self.format = format

#   def set_time(self, time: float):
#     self.time = time
  

